<a href="https://colab.research.google.com/github/skwh54/gemma-sprint/blob/main/diary_choding_with_Title_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install peft
! pip install trl
! pip install bitsandbytes
! pip install deepspeed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 14.4 MB/s eta 0:00:00

In [2]:
import os
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/filtered_smile.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['formal', 'choding'],
        num_rows: 3470
    })
})

In [ ]:
BASE_MODEL = "google/gemma-2-2b-it"


model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset['train']["formal"][0]

'안녕하세요. 저는 고양이 6마리 키워요.'

In [ ]:
dataset['train']["choding"][0]

'ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ'

In [ ]:
doc = dataset['train']["formal"][0]
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
messages = [
    {
        "role": "user",
        "content": "다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:\n\n안녕하세요. 저는 고양이 6마리 키워요.<end_of_turn>\n<start_of_turn>model\n'

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

사랑하는 고양이 6마리가 있어요! 😻😻😻😻😻😻 



In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['formal'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['formal'][i], example['choding'][i]))
    return prompt_list

In [ ]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:

안녕하세요. 저는 고양이 6마리 키워요.<end_of_turn>
<start_of_turn>model
ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ<end_of_turn><eos>


In [ ]:
torch.cuda.is_available()

True

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# bnb_config = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
BASE_MODEL = "google/gemma-2-2b-it"

#model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", load_in_8bit=True,quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'



config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##################################### (eager 추가한 버전)

BASE_MODEL = "google/gemma-2-2b-it"

# 모델 및 토크나이저 불러오기 (attn_implementation='eager' 추가 및 양자화 설정 유지)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="eager"  # 'eager' attention 적용
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'
##########################################

In [ ]:
train_data = dataset['train']

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=256,
    args=TrainingArguments(
        output_dir="/content/output",
#        num_train_epochs = 1,
        max_steps=1000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        optim="paged_adamw_8bit",
        warmup_steps=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3470 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   3366 MiB |   4461 MiB |   8353 MiB |   4986 MiB |
|       from large pool |   3313 MiB |   4438 MiB |   8299 MiB |   4986 MiB |
|       from small pool |     52 MiB |     52 MiB |     53 MiB |      0 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   3366 MiB |   4461 MiB |   8353 MiB |   4986 MiB |
|       from large pool |   3313 MiB |   4438 MiB |   8299 MiB |

In [ ]:
trainer.train()

[2024-09-26 14:37:11,438] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
100,1.558900
200,1.042700
300,1.018100
400,0.969200
500,0.876900
600,0.820400
700,0.824500
800,0.800000
900,0.775100
1000,0.684000


TrainOutput(global_step=1000, training_loss=0.9369793167114258, metrics={'train_runtime': 3040.1434, 'train_samples_per_second': 2.631, 'train_steps_per_second': 0.329, 'total_flos': 6478863637768704.0, 'train_loss': 0.9369793167114258, 'epoch': 2.3054755043227666})

In [ ]:
ADAPTER_MODEL = "/content/output/checkpoint-1000/lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('/content/gemma-2-2b-it-choding')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Test


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
BASE_MODEL = "google/gemma-2-2b-it"
#FINETUNE_MODEL = "/content/gemma-2-2b-it-choding"

FINETUNE_MODEL = "/content/drive/MyDrive/gemma-sprint/gemma-2-2b-it-choding"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)


In [ ]:
doc = dataset['train']['formal'][13]

In [ ]:
doc = "저는 외교관 되는게 꿈이에요."

In [ ]:
doc = "안녕하세요. 저는 친구와 공원에서 만나서 놀았어요. 날씨가 좋아서 놀기 좋았어요. 맛있는 음식들도 많이 먹었어요. 내일도 놀고 싶다!"

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:\n\n{}".format(doc)
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 일기를 초등학생이 쓴 것처럼 요약하고 일기를 변환해주세요:\n\n{}\n\n제목: (내용요약)\n내용:".format(doc)
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

NameError: name 'doc' is not defined

In [ ]:
prompt

'<bos><start_of_turn>user\n다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:\n\n안녕하세요. 저는 친구와 공원에서 만나서 놀았어요. 날씨가 좋아서 놀기 좋았어요. 맛있는 음식들도 많이 먹었어요. 내일도 놀고 싶다!<end_of_turn>\n<start_of_turn>model\n'

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

ㅎㅇ 난 친구랑 공원에서 만나서 놀았는데 날씨가 좋아서 더 좋았고 맛있는 음식도 많이 먹었음 내일도 놀고 싶다


Title 학습


In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/choding_diary.csv")

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'sentence'],
        num_rows: 1951
    })
})

In [ ]:
BASE_MODEL = "/content/drive/MyDrive/gemma-sprint/gemma-2-2b-it-choding"


model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/gemma-sprint/gemma-2-2b-it-choding"))

['config.json', 'generation_config.json', 'model-00001-of-00002.safetensors', 'model-00002-of-00002.safetensors', 'model.safetensors.index.json']


In [ ]:
dataset['train']["sentence"][0]

'5학년을 마치며....                                       2020. 2. 19 5학년은 나에게 기쁘고 힘든 추억이다. 기쁘날이 더 많았지만, 힘든날도 있었다. 5학년 친구들 반은 거의 모르는 친구었다. 같이 놀고 장난치고 그런걸 5학년때 제일 많이 한것같다. 우리반은 선생님이 2분이셨지만, 2분다 좋은 선생님이 었다. 우리반 친구들은 미안한거, 서운한거, 기쁜게 많지만 항상 같이 놀아 주셔서 고맙다. 아직까지는 6학년이 되기 싫다. 반 배정, 공부, 새로운 친구와 선생님 , 시험를 생각하면 싫다. 그렇지만 다른 기쁜걸 생각하면 빨리 되고 싶다 . 5-3반은 기억할 것이다. 5학년이 마쳐서 아쉬웠지만 괜찮다. 화이팅 -끝-'

In [ ]:
dataset['train']["title"][0]

'  5학년을 마치며....   '

In [ ]:
doc = dataset['train']["sentence"][0]
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
messages = [
    {
        "role": "user",
        "content": "다음 일기의 제목을 작성해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

NameError: name 'model' is not defined

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [ ]:
tokenizer.padding_side = 'right'

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

5학년은 기쁘고 힘든 추억이였어요


In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['sentence'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 일기의 제목을 작성해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['sentence'][i], example['title'][i]))
    return prompt_list

In [ ]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 일기의 제목을 작성해주세요:

5학년을 마치며....                                       2020. 2. 19 5학년은 나에게 기쁘고 힘든 추억이다. 기쁘날이 더 많았지만, 힘든날도 있었다. 5학년 친구들 반은 거의 모르는 친구었다. 같이 놀고 장난치고 그런걸 5학년때 제일 많이 한것같다. 우리반은 선생님이 2분이셨지만, 2분다 좋은 선생님이 었다. 우리반 친구들은 미안한거, 서운한거, 기쁜게 많지만 항상 같이 놀아 주셔서 고맙다. 아직까지는 6학년이 되기 싫다. 반 배정, 공부, 새로운 친구와 선생님 , 시험를 생각하면 싫다. 그렇지만 다른 기쁜걸 생각하면 빨리 되고 싶다 . 5-3반은 기억할 것이다. 5학년이 마쳐서 아쉬웠지만 괜찮다. 화이팅 -끝-<end_of_turn>
<start_of_turn>model
  5학년을 마치며....   <end_of_turn><eos>


In [ ]:
torch.cuda.is_available()

True

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# bnb_config = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
BASE_MODEL = "/content/drive/MyDrive/gemma-sprint/gemma-2-2b-it-choding"

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)

# 토크나이저 로드 (add_special_tokens=True 제거)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# padding_side 설정
tokenizer.padding_side = 'right'

# 필요한 경우 특수 토큰 추가
special_tokens_dict = {'pad_token': '[PAD]', 'sep_token': '[SEP]', 'cls_token': '[CLS]'}  # 예시
tokenizer.add_special_tokens(special_tokens_dict)

# 모델에도 특수 토큰 크기 반영 (토크나이저와 모델이 동일하게 반영되도록)
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding(256003, 2304, padding_idx=0)

In [ ]:
#train_data = dataset['train']

In [ ]:

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=256,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/gemma-sprint/gemma-2-2b-it-choding-withtitle",
#        num_train_epochs = 1,
        max_steps=1000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        optim="paged_adamw_8bit",
        warmup_steps=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/acce

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   5578 MiB |   6672 MiB |  15550 MiB |   9972 MiB |
|       from large pool |   5502 MiB |   6627 MiB |  15474 MiB |   9972 MiB |
|       from small pool |     75 MiB |     75 MiB |     75 MiB |      0 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   5578 MiB |   6672 MiB |  15550 MiB |   9972 MiB |
|       from large pool |   5502 MiB |   6627 MiB |  15474 MiB |

In [ ]:
trainer.train()

[2024-09-28 15:24:27,862] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
100,2.456000
200,2.270600
300,2.133000
400,2.056300
500,1.999000
600,1.869000
700,1.844100
800,1.744700
900,1.674000
1000,1.668200


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=1000, training_loss=1.9714951171875, metrics={'train_runtime': 4383.6052, 'train_samples_per_second': 1.825, 'train_steps_per_second': 0.228, 'total_flos': 1.899623504211149e+16, 'train_loss': 1.9714951171875, 'epoch': 4.1004613018964635})

In [ ]:
ADAPTER_MODEL = "/content/output/checkpoint-1000/lora_adapter_title"

trainer.model.save_pretrained(ADAPTER_MODEL)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
# 기본 모델 로드
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)

# 어댑터 모델에 맞게 기본 모델의 임베딩 크기 조정 (256003으로 맞춤)
model.resize_token_embeddings(256003)

# 어댑터 모델 로드
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

# 모델 병합 및 언로드
model = model.merge_and_unload()

# 모델 저장
model.save_pretrained('/content/gemma-2-2b-it-choding-withtitle')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fianl Test

In [5]:
BASE_MODEL = "google/gemma-2-2b-it"


#FINETUNE_MODEL = "/content/drive/MyDrive/gemma-sprint/gemma-2-2b-it-choding"
FINETUNE_MODEL = "skwh54/Gemma-2-2b-it-choding-w.title"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(FINETUNE_MODEL)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.1k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)


In [ ]:
doc = dataset['train']['formal'][13]

In [ ]:
doc = dataset['train']['formal'][13]

In [24]:
doc = "저는 외교관 되는게 꿈이에요."

In [31]:
doc = "안녕하세요. 저는 친구와 공원에서 만나서 놀았어요. 날씨가 좋아서 놀기 좋았어요. 맛있는 음식들도 많이 먹었어요. 내일도 놀고 싶다!"

In [27]:
doc = "오늘은 바쁜 하루였다. 아침에 일어나 출근 준비를 하면서 커피 한 잔으로 잠을 깨웠다. 동료들과 함께 점심을 먹으며 잠시 여유를 찾았다. 퇴근 후에는 헬스장에 가서 운동을 했고, 집에 와서는 간단한 저녁을 먹고 TV를 보며 쉬었다. 이제 잠자리에 들 준비를 하며 내일을 기대해본다."

In [32]:
messages = [
    {
        "role": "user",
        "content": "다음 일기의 제목을 작성해주세요.:\n\n{}".format(doc)
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [33]:
messages2 = [
    {
        "role": "user",
        "content": "다음 일기를 초등학생이 쓴 것처럼 바꿔주세요:\n\n{}".format(doc)
    }
]
prompt2 = pipe_finetuned.tokenizer.apply_chat_template(messages2, tokenize=False, add_generation_prompt=True)

In [9]:
prompt

'<bos><start_of_turn>user\n다음 일기의 제목을 작성해주세요. 다음 일기를 초등학생이 쓴 것처럼 변환해주세요:\n\n오늘은 바쁜 하루였다. 아침에 일어나 출근 준비를 하면서 커피 한 잔으로 잠을 깨웠다. 동료들과 함께 점심을 먹으며 잠시 여유를 찾았다. 퇴근 후에는 헬스장에 가서 운동을 했고, 집에 와서는 간단한 저녁을 먹고 TV를 보며 쉬었다. 이제 잠자리에 들 준비를 하며 내일을 기대해본다.<end_of_turn>\n<start_of_turn>model\n'

In [34]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

outputs2 = pipe_finetuned(
    prompt2,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])
print(outputs2[0]["generated_text"][len(prompt):])

공원에 간 날
model
안녕 나는 친구랑 공원에서 놀았어
